# Initialize

In [ ]:
%load_ext ipycache
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

sys.path.insert(0, '../../')


adsorbate = 'CO'
target_energy = -0.67
initial_training_size = 1000
batch_size = 200
quantile_cutoff = 0.95

In [ ]:
import random
import ase.db


db_dir = '../pull_data/%s/' % adsorbate
db = ase.db.connect(db_dir + '%s.db' % adsorbate)
rows = list(db.select())
random.Random(42).shuffle(rows)


def parse_rows(rows):
    features = []
    labels = []
    surfaces = []

    for row in rows:
        features.append(row.id)
        data = row.data
        labels.append(data['adsorption_energy'])
        surface = (data['mpid'], data['miller'], data['shift'], data['top'])
        surfaces.append(surface)

    return features, labels, surfaces


training_features, training_labels, training_surfaces = parse_rows(rows[:initial_training_size])
sampling_features, sampling_labels, sampling_surfaces = parse_rows(rows[initial_training_size:])

# Random Sampling Procedures

### Random Sampling with Null Model

In [ ]:
cd ../random/null

In [ ]:
sys.path.insert(0, '../../..')

from src.discoverers.adsorption.models import NullModel
from src.discoverers.adsorption.randomsearch import RandomSearcher
from src.discoverers.adsorption.values import calc_co2rr_activities

db_dir = '../../pull_data/%s/' % adsorbate

model = NullModel(db_dir, quantile_cutoff)
rs_null_discoverer = RandomSearcher(model=model,
                                    quantile_cutoff=quantile_cutoff,
                                    value_calculator=calc_co2rr_activities,
                                    batch_size=batch_size,
                                    training_features=training_features,
                                    training_labels=training_labels,
                                    training_surfaces=training_surfaces,
                                    sampling_features=sampling_features,
                                    sampling_labels=sampling_labels,
                                    sampling_surfaces=sampling_surfaces,
                                    #n_samples=3,  # decrease to speed up
                                    #init_train=False  # Set to `False` only for warm starts
                                    )

# rs_null_discoverer.simulate_discovery()

rs_null_discoverer.load_last_run()

rs_null_discoverer.plot_performance(window=100)

### Random sampling with CFGP (NOTE: uses prime for now until I cache files)

In [ ]:
cd ../prime

In [ ]:
from src.discoverers.adsorption.models import PrimeModel


model = PrimeModel(db_dir)
rs_cfgp_discoverer = RandomSearcher(model=model,
                                    quantile_cutoff=quantile_cutoff,
                                    value_calculator=calc_co2rr_activities,
                                    batch_size=batch_size,
                                    training_features=training_features,
                                    training_labels=training_labels,
                                    training_surfaces=training_surfaces,
                                    sampling_features=sampling_features,
                                    sampling_labels=sampling_labels,
                                    sampling_surfaces=sampling_surfaces,
                                    #n_samples=3,  # decrease to speed up
                                    #init_train=False  # Set to `False` only for warm starts
                                    )

# rs_cfgp_discoverer.simulate_discovery()

rs_cfgp_discoverer.load_last_run()

rs_cfgp_discoverer.plot_performance(window=100)

# MMS Procedures

### MMS with null model

In [ ]:
cd ../../MMS/null

In [ ]:
sys.path.insert(0, '../../..')

from src.discoverers.adsorption.mms import MultiscaleDiscoverer
from src.discoverers.adsorption.models import NullModel


db_dir = '../../pull_data/%s/' % adsorbate

model = NullModel(db_dir, quantile_cutoff)
mms_null_discoverer = MultiscaleDiscoverer(model=model,
                                           quantile_cutoff=quantile_cutoff,
                                           value_calculator=calc_co2rr_activities,
                                           batch_size=batch_size,
                                           training_features=training_features,
                                           training_labels=training_labels,
                                           training_surfaces=training_surfaces,
                                           sampling_features=sampling_features,
                                           sampling_labels=sampling_labels,
                                           sampling_surfaces=sampling_surfaces,
                                           #n_samples=3,  # decrease to speed up
                                           init_train=False  # Set to `False` only for warm starts
                                          )

# mms_null_discoverer.simulate_discovery()

mms_null_discoverer.load_last_run()

mms_null_discoverer.plot_performance(window=100)

### MMS with CFGP

In [ ]:
cd ../CFGP

In [ ]:
import os
os.environ['PYTHONPATH'] = '/home/jovyan/GASpy:/home/jovyan/GASpy/GASpy_regressions:'

import sys
sys.path.insert(0, '/home/jovyan/GASpy')
sys.path.insert(0, '/home/jovyan/GASpy/GASpy_regressions')

sys.path.append('../../..')
from src.discoverers.adsorption.mms import MultiscaleDiscoverer
from src.discoverers.adsorption.models import CFGP
from src.discoverers.adsorption.values import calc_co2rr_activities


# Initialize
model = CFGP(db_dir)
mms_cfgp_discoverer = MultiscaleDiscoverer(model=model,
                                           value_calculator=calc_co2rr_activities,
                                           quantile_cutoff=quantile_cutoff,
                                           batch_size=batch_size,
                                           training_features=training_features,
                                           training_labels=training_labels,
                                           training_surfaces=training_surfaces,
                                           sampling_features=sampling_features,
                                           sampling_labels=sampling_labels,
                                           sampling_surfaces=sampling_surfaces,
                                           #init_train=False  # Set to `False` only for warm starts
                                           )

# mms_cfgp_discoverer.simulate_discovery()

mms_cfgp_discoverer.load_last_run()

mms_cfgp_discoverer.plot_performance(window=100)

# Comparison

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
import pandas as pd


FORMATTER = ticker.FuncFormatter(lambda x, p: format(int(x), ','))
FIG_SIZE = (6.5, 2.5)


def plot_rolling_metric(ax, metric_values, metric_name, label=None,
                        window=200, smoother='mean', unit=''):
    '''
    Helper function to plot model performance metrics across time in
    hallucination.

    Args:
        ax              Matplotlib ax object to plot onto
        metric_values   A sequence of floats that will be plotted against
                        batch number in the hallucination.
        metric_name     A string indicating what you want the values to be
                        labeled as in the plots.
        label           A string indicating the label you want to use for
                        the data
        window          How many points to roll over during each iteration
        smoother        String indicating how you want to smooth the
                        residuals over the course of the hallucination.
                        Corresponds exactly to the methods of the
                        `pandas.DataFrame.rolling` class, e.g., 'mean',
                        'median', 'min', 'max', 'std', 'sum', etc.
        unit            [Optional] String indicating the units you want to
                        label the plot with
    '''
    # Format the data
    df = pd.DataFrame(metric_values, columns=[metric_name])
    rolling_residuals = getattr(df, metric_name).rolling(window=window)
    rolled_values = getattr(rolling_residuals, smoother)().values
    query_numbers = list(range(len(rolled_values)))

    # Create and format the figure
    _ = sns.lineplot(query_numbers, rolled_values, ax=ax, label=label)
    _ = ax.set_xlabel('Number of discovery queries')
    if unit:
        unit = ' [' + unit + ']'
    _ = ax.set_ylabel('Rolling %s of \n%s%s' % (smoother, metric_name, unit))
    _ = ax.set_xlim([query_numbers[0], query_numbers[-1]])
    _ = ax.set_ylim([0., np.nanmax(rolled_values) * 1.1])
    _ = fig.set_size_inches(*FIG_SIZE)
    _ = ax.get_xaxis().set_major_formatter(FORMATTER)
    return fig

### F1 Score

In [ ]:
fig = plt.figure()

ax_mms = sns.scatterplot(list(range(len(mms_discoverer.reward_history))),
                         mms_discoverer.reward_history,
                         label='MMS', marker='1')
ax_rs = sns.scatterplot(list(range(len(rs_discoverer.reward_history))),
                        rs_discoverer.reward_history,
                        label='RS w/ CFGP', marker='+')
ax_tpot = sns.scatterplot(list(range(len(tpot_discoverer.reward_history))),
                          tpot_discoverer.reward_history,
                          label='GASpy (old)', marker='x')
ax_null = sns.scatterplot(list(range(len(null_discoverer.reward_history))),
                          null_discoverer.reward_history,
                          label='RS w/ $∅$', marker='.')

_ = ax_mms.set_xlabel('batch number')
_ = ax_mms.set_ylabel('F1 score')
_ = ax_mms.set_xlim(0, 100)
_ = ax_mms.set_ylim(0., 1.2)
_ = fig.set_size_inches(*FIG_SIZE)
_ = ax_mms.get_xaxis().set_major_formatter(FORMATTER)

### Residuals

In [ ]:
fig = plt.figure()
ax = fig.gca()

_ = plot_rolling_metric(ax=ax,
                        metric_values=np.abs(mms_discoverer.residuals),
                        label='MMS',
                        metric_name='|residuals|',
                        unit='eV')
_ = plot_rolling_metric(ax=ax,
                        metric_values=np.abs(rs_discoverer.residuals),
                        label='random search with CFGP',
                        metric_name='|residuals|',
                        unit='eV')
_ = plot_rolling_metric(ax=ax,
                        metric_values=np.abs(tpot_discoverer.residuals),
                        label='GASpy heuristic',
                        metric_name='|residuals|',
                        unit='eV')

_ = ax.set_ylim([0., 0.8])

### Sharpness

In [ ]:
fig = plt.figure()
ax = fig.gca()

_ = plot_rolling_metric(ax=ax,
                        metric_values=mms_discoverer.uncertainties,
                        label='MMS',
                        metric_name='predicted uncertainty (stdev)',
                        unit='eV')
_ = plot_rolling_metric(ax=ax,
                        metric_values=rs_discoverer.uncertainties,
                        label='random search with CFGP',
                        metric_name='predicted uncertainty (stdev)',
                        unit='eV')
#_ = plot_rolling_metric(ax=ax,
#                        metric_values=tpot_discoverer.uncertainties,
#                        label='GASpy heuristic',
#                        metric_name='predicted uncertainty (stdev)',
#                        unit='eV')

_ = ax.set_ylim([0., 1.])

### Calibration

In [ ]:
from tqdm.notebook import tqdm


fig = plt.figure()
ax = fig.gca()


def calculate_eces(discoverer, window=200):
    # Divide the data into chunks, which we need to calculate ECE
    chunked_residuals = discoverer.chunk_iterable(discoverer.residuals, window)
    chunked_uncertainties = discoverer.chunk_iterable(discoverer.uncertainties, window)

    # Calculate ECE
    loop = tqdm(zip(chunked_residuals, chunked_uncertainties),
                desc='calibration', unit='batch', total=len(chunked_residuals))
    for resids, stdevs in loop:
        ece = discoverer.calculate_expected_calibration_error(resids, stdevs)
        try:
            eces.extend([ece] * len(resids))
        # EAFP for initialization
        except NameError:
            eces = [ece] * len(resids)
    return eces


_ = plot_rolling_metric(ax=ax,
                        metric_values=calculate_eces(mms_discoverer),
                        label='MMS',
                        metric_name='expected calibration error')
_ = plot_rolling_metric(ax=ax,
                        metric_values=calculate_eces(rs_discoverer),
                        label='random search with CFGP',
                        metric_name='expected calibration error')
#_ = plot_rolling_metric(ax=ax,
#                        metric_values=calculate_eces(tpot_discoverer),
#                        label='GASpy heuristic',
#                        metric_name='expected calibration error')

_ = ax.set_ylim([0., 0.05])

### NLL

In [ ]:
from scipy.stats import norm


fig = plt.figure()
ax = fig.gca()


def calculate_nlls(discoverer):
    nlls = [-norm.logpdf(resid, loc=0., scale=std)
            for resid, std in zip(discoverer.residuals, discoverer.uncertainties)]
    return nlls


_ = plot_rolling_metric(ax=ax,
                        metric_values=calculate_nlls(mms_discoverer),
                        label='MMS',
                        metric_name='negative log likelihood',
                        window=400)
_ = plot_rolling_metric(ax=ax,
                        metric_values=calculate_nlls(rs_discoverer),
                        label='random search with CFGP',
                        metric_name='negative log likelihood',
                        window=400)
#_ = plot_rolling_metric(ax=ax,
#                        metric_values=calculate_nlls(tpot_discoverer),
#                        label='GASpy heuristic',
#                        metric_name='negative log likelihood',
#                        window=400)

_ = ax.set_ylim([-0.5, 3.])